In [15]:
import math
import numpy as np

epsilon = 1e-5

def slope(p1,p2):
    return (p2[1]-p1[1])/(p2[0]-p1[0] + epsilon)

def above_line(p, st, slope):
    y = (p[0] - st[0]) * slope + st[1]
    return p[1] < y

def reorder_points(points):
    "Reorder points in clockwise order from top left = i.e. (tl, tr, br, bl)"
    
    #Point with minimum x
    ordered_points = sorted(points, key=lambda x: (x[0], x[1]))
    p1 = ordered_points[0]
    
    #Find 3rd point with middle slop
    slopes = [[slope(p1, p), p] for p in ordered_points[1:]]
    ordered_slopes = sorted(slopes, key=lambda x: x[0])
    slope_13, p3 = ordered_slopes[1]
    
    #Find 2nd point above 1-3 diagonal - guarantees clockwise now
    if above_line(ordered_slopes[0][1], p3, slope_13):
        p2 = ordered_slopes[0][1]
        p4 = ordered_slopes[2][1]
        reverse = False
    else:
        p2 = ordered_slopes[2][1]
        p4 = ordered_slopes[0][1]
        reverse = True

    # Find the top left point.
    slope_24 = slope(p2, p4)
    if slope_13 < slope_24:
        if reverse:
            reorder_points = [p4, p1, p2, p3]
        else:
            reorder_points = [p2, p3, p4, p1]
    else:
        reorder_points = [p1, p2, p3, p4]

    return reorder_points

def min_box_dist(box1, box2):
    box_dist = [math.pow((p2[0] - p1[0]), 2) + math.pow((p2[1] - p1[1]), 2) for p1 in box1 for p2 in box2]
    return np.min(box_dist)

def reorder_box(boxes):
    "Reorder character bounding boxes to get adjacent boxes"
    
    #Compute distance matrix between every pair of boxes
    n = len(boxes)
    M = np.zeros((n,n), dtype = np.float32)
    for i in range(n):
        box1 = boxes[i]
        for j in range(i+1,n):
            box2 = boxes[j]
            dist = min_box_dist(box1,box2)
            M[i][j]=dist
            M[j][i]=dist
    
    #Get boxes at extreme ends
    end_ind = np.argmax(M)
    inf = M[end_ind//n, end_ind%n] + 1 #Index from flattened matrix index
    for i in range(n):
        M[i,i] = inf
    
    cur_ind = end_ind//n
    reordered_boxes = [boxes[cur_ind]]
    for i in range(n-1):
        M[:,cur_ind] = inf #To not repeat box
        closest_box = np.argmin(M[cur_ind])
        reordered_boxes.append(boxes[closest_box])
        cur_ind = closest_box
        
    return reordered_boxes
        
def tri_area(p1,p2,p3):
    return abs((p2[0] - p1[0]) * (p3[1] - p1[1]) - (p3[0] - p1[0]) * (p2[1] - p1[1])) / 2

def quad_area(p1,p2,p3,p4):
    p1,p2,p3,p4 = reorder_points([p1,p2,p3,p4])
    s1 = tri_area(p1,p2,p3)
    s2 = tri_area(p1,p3,p4)
    s3 = tri_area(p1,p2,p4)
    s4 = tri_area(p2,p3,p4)
    
    #Check if an actual convex rectangle is being formed 
    if s1 + s2 == s3 + s4:
        return s1 + s2
    else:
        return 0

def center(p1,p2,p3):
    "Calculate center of triangle"
    points = np.array([p1,p2,p3])
    return [round(np.average(points[:,0])), round(np.average(points[:,1]))]

def intersect(points):
    "Calculate the center of quadrilateral - intersection of diagonals"
    [x1, y1], [x2, y2], [x3, y3], [x4, y4] = points
    x = ((x3 - x1) * (x4 - x2) * (y2 - y1) + x1 * (y3 - y1) * (x4 - x2) - x2 * (y4 - y2) * (x3 - x1)) \
        / ((y3 - y1) * (x4 - x2) - (y4 - y2) * (x3 - x1) + epsilon)
    y = (y3 - y1) * ((x4 - x2) * (y2 - y1) + (x1 - x2) * (y4 - y2)) \
        / ((y3 - y1) * (x4 - x2) - (y4 - y2) * (x3 - x1) + epsilon) + y1
    return [x, y]

def cal_ppairs(points):
    "Calculate possible vertices of affinity box"
    box_c = intersect(points)
    p1,p2,p3,p4 = points
    return [[center(p1,p2,box_c), center(p3,p4,box_c)], [center(p1,p3,box_c), center(p2,p4,box_c)]]

def cal_abox(pairs1,pairs2):
    areas = [quad_area(p1,p2,p3,p4) for p1,p2 in pairs1 for p3,p4 in pairs2]
    #Check which pairs of points gives an actual convex quadrilateral
    max_ind = np.argmax(np.array(areas))
    
    abox = [pairs1[max_ind//2][0], pairs1[max_ind//2][1], pairs2[max_ind%2][0], pairs2[max_ind%2][1]]
    abox = reorder_points(abox)
    return abox

def cal_affinity_boxes(region_boxes):
    
    #Reorder boxes to get pairs of adjacent boxes
    region_boxes = reorder_box([reorder_points(r_box) for r_box in region_boxes])
    
    #Get point pairs in each character level region
    point_pairs = [cal_ppairs(r_box) for r_box in region_boxes] 
    affinity_boxes = []
    
    #Get affinity box for each pair of adjacent point pairs
    for i in range(len(point_pairs)-1):
        a_box = reorder_points(cal_abox(point_pairs[i],point_pairs[i+1]))
        affinity_boxes.append(a_box)
    return affinity_boxes

(251, 96)
(284, 112)
(267, 112)
(253, 118)

Reordered Points
 [[251, 96], [284, 112], [267, 112], [253, 118]]

Reordered Points
 [[0, 0], [4, 0], [4, 4], [0, 4]]

Reordered Points
 [[15, 45], [56, 25], [85, 45], [25, 80]]

Reordered Boxes
 [[[0, 0], [4, 4], [4, 0], [0, 4]], [[4, 0], [8, 4], [8, 0], [4, 4]], [[8, 0], [12, 4], [12, 0], [8, 4]], [[12, 0], [16, 4], [16, 0], [12, 4]], [[16, 0], [20, 4], [20, 0], [16, 4]]]

Point Pairs

[[[2, 1], [2, 3]], [[2, 2], [2, 2]]]
[[[6, 1], [6, 3]], [[6, 2], [6, 2]]]
[[[10, 1], [10, 3]], [[10, 2], [10, 2]]]
[[[14, 1], [14, 3]], [[14, 2], [14, 2]]]
[[[18, 1], [18, 3]], [[18, 2], [18, 2]]]



Affinity Boxes
 [[[2, 1], [6, 1], [6, 3], [2, 3]], [[6, 1], [10, 1], [10, 3], [6, 3]], [[10, 1], [14, 1], [14, 3], [10, 3]], [[14, 1], [18, 1], [18, 3], [14, 3]]]
